In [1]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import chart_studio.plotly as py
import plotly.graph_objs as go
from sklearn import svm
from sklearn.metrics import roc_auc_score, log_loss
from sklearn.model_selection import StratifiedKFold
from methods import *
import torch
import pickle, os

In [2]:
%load_ext autoreload
%autoreload 2

In [65]:
def get_predicted_label(model, image, device):
    output = model(image.to(device))
    softmax = nn.functional.softmax(out[0].cpu(), dim=0)
    pred_label = out.max(1)[1].item()
    return label

def save_obj(obj, filename):
    with open(filename, 'wb') as handle:
        pickle.dump(obj, handle)

def load_obj(filename):
    if not os.path.isfile(filename):
        print('Pickle {} does not exist.'.format(filename))
        return None
    with open(filename, 'rb') as handle:
        obj = pickle.load(handle)
    return obj

def read_features(p_path, trigger_type_aux_str=None):
    """trigger_type_aux_str: used to select only rows from models with polygon or filter backdoors. If none, all data is chosen"""
    report = pd.read_csv(p_path)
    if trigger_type_aux_str is None:
        print('Using all rows (clean, polygon and instagram filters)')
    else:
        print(f'Using only clean rows and {trigger_type_aux_str}-backdoored rows from the dataset')
        indexes = []
        for i in range(len(report)):
            col = report['trigger_type_aux'].iloc[i]
            if (trigger_type_aux_str in col.lower()) or (col.lower() == 'none'):
                indexes.append(i)
        report = report.iloc[indexes]
    initial_columns = report.columns
    col_model_label = report['model_label'].copy(deep=True)
    for c in initial_columns:
        if not c.endswith('mean_diff') and not c.endswith('std_diff'):
            del report[c]
    features = report.values
    labels = np.array([int(col_model_label.iloc[i] == 'backdoor') for i in range(len(report))])
    return abs(features), labels

def read_features_confusion_matrix(p_path, trigger_type_aux_str=None):
    """trigger_type_aux_str: used to select only rows from models with polygon or filter backdoors. If none, all data is chosen"""
    report = pd.read_csv(p_path)
    initial_columns = report.columns
    col_model_label = report['model_label'].copy(deep=True)
    for c in initial_columns:
        if not c.startswith('h_') and not c.startswith('kl_'):
            del report[c]
    features = report.values
    labels = np.array([int(col_model_label.iloc[i] == 'backdoor') for i in range(len(report))])
    return abs(features), labels

def evaluate_classifier(train_x, train_y, test_x, test_y):
#     clf = svm.SVC(kernel='linear', probability=True)
    clf = svm.SVC(C=10, kernel='rbf', gamma='scale', probability=True)
#     clf = svm.SVC(kernel='rbf', gamma='auto', probability=True)
    clf.fit(train_x, train_y)
    y_score = clf.predict(test_x)
    y_pred = clf.predict_proba(test_x)
    roc_auc = roc_auc_score(y_true=test_y, y_score=y_score)
    cross_entropy = log_loss(y_true=test_y, y_pred=y_pred)
    return roc_auc, cross_entropy

# stratified K-fold validation for a full training dataset (square-size and 5 filters)

In [66]:
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square25-filters_triggered_classes.csv' # old dataset
path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
# path_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'

n_splits = 5
trigger_type_aux_str = None
print(f'trigger_type_aux_str={trigger_type_aux_str}')

if 'confusion-matrix' in path_csv:
    X, y = read_features_confusion_matrix(path_csv, trigger_type_aux_str)
else:
    X, y = read_features(path_csv, trigger_type_aux_str) # clean data is automatically added

skf = StratifiedKFold(n_splits=n_splits, shuffle=True)
scores_roc, scores_xent = [], []

for index, (train_index, test_index) in enumerate(skf.split(X, y)):
    X_train, y_train = X[train_index, :], y[train_index]
    X_test, y_test = X[test_index, :], y[test_index]
    
    roc, xent = evaluate_classifier(X_train, y_train, X_test, y_test)
    
    scores_roc.append(roc)
    scores_xent.append(xent)

print(f'Stratified {n_splits}-fold cross-validation')
print(f'avg roc ={sum(scores_roc) / len(scores_roc)}')
print(f'std roc ={np.std(scores_roc)}')
print()
print(f'avg xent={sum(scores_xent) / len(scores_xent)}')
print(f'std roc ={np.std(scores_xent)}')

trigger_type_aux_str=None
Using all rows (clean, polygon and instagram filters)
Stratified 5-fold cross-validation
avg roc =0.7808435708435708
std roc =0.025791236961583336

avg xent=0.46533106363993415
std roc =0.03268973017746643


# Local Testing: train on training data and test on holdout data

In [67]:
path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray.csv'
path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray.csv'
# path_train_csv = r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
# path_holdout_csv = r'confusion-reports\ics_svm\round2-holdout-dataset\round2-holdout-dataset_square-25-filters_all-classes_gray_confusion-matrix.csv'
trigger_type_aux_str = None

print('Local Testing')
if 'confusion-matrix' in path_train_csv and 'confusion-matrix' in path_holdout_csv:
    print('Approach: confusion matrix and original CNN')
    X_train, y_train = read_features_confusion_matrix(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features_confusion_matrix(path_holdout_csv, trigger_type_aux_str)
else:
    print('Approach: confusion distribution and SDNs')
    X_train, y_train = read_features(path_train_csv, trigger_type_aux_str)
    X_holdout, y_holdout = read_features(path_holdout_csv, trigger_type_aux_str)

print('train shape:', X_train.shape, y_train.shape)
print('holdout shape:', X_holdout.shape, y_holdout.shape)

roc, xent = evaluate_classifier(X_train, y_train, X_holdout, y_holdout)
print(f'ROC AUC = {roc}')
print(f'Cross-Entropy = {xent}')

Local Testing
Approach: confusion distribution and SDNs
Using all rows (clean, polygon and instagram filters)
Using all rows (clean, polygon and instagram filters)
train shape: (1104, 12) (1104,)
holdout shape: (144, 12) (144,)
ROC AUC = 0.8055555555555556
Cross-Entropy = 0.4540606428296601


# Train meta-model using square data and filters

In [ ]:
X_train, y_train = read_features(r'confusion-reports/ics_svm/round2-train-dataset/round2-train-dataset_square25-filters.csv', None)
model = svm.SVC(kernel='linear', probability=True)
model.fit(X_train, y_train)
save_obj(model, r'..\metamodel_svm_square25_filters_black_square.pickle')
print('done')

# Plot trigger-size vs ROC using 5-fold cross validation for Square Data

In [ ]:
list_square_sizes = [5, 10, 15, 20, 25, 30, 35, 40, 45, 50]
dict_square_sizes = {size:index*2 for index, size in enumerate(list_square_sizes)}
roc_auc_scores, roc_auc_err = [], []
xent_scores, xent_err = [], []
trigger_type_aux_str = 'polygon'
print(f'trigger_type_aux_str = {trigger_type_aux_str}\n')
for size, index in dict_square_sizes.items():
    path = 'confusion-reports/ics_svm/round2-train-dataset/round2-train-dataset_squares.csv'
    X, y = read_features(path, trigger_type_aux_str)
    
    print(f'selecting columns {index} and {index+1}')
    X = X[:, index:index+2]
    
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    cv_scores_roc, cv_scores_xent = [], []
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, y_train = X[train_index, :], y[train_index] # square_diffs have indexes 0,1 !!!
        X_test, y_test = X[test_index, :], y[test_index] # square_diffs have indexes 0,1 !!!
        roc, xent = evaluate_classifier(X_train, y_train, X_test, y_test)
        cv_scores_roc.append(roc)
        cv_scores_xent.append(xent)
    mean_roc, std_roc = np.mean(cv_scores_roc), np.std(cv_scores_roc)
    mean_xent, std_xent = np.mean(cv_scores_xent), np.std(cv_scores_xent)
    print(f'size={size}, index={index}: mean_roc={mean_roc}, std_roc={std_roc}, mean_xent={mean_xent}, std_xent={std_xent}\n')
    
    roc_auc_scores.append(mean_roc)
    roc_auc_err.append(std_roc)
    xent_scores.append(mean_xent)
    xent_err.append(std_xent)

plt.figure(figsize=(10, 6)).patch.set_color('white')
plt.errorbar(list_square_sizes, roc_auc_scores, roc_auc_err, marker='o', label='all models')
plt.grid()
plt.legend()
plt.xlabel('square trigger size')
plt.ylabel('ROC AUC for SVM classifier')
plt.xticks(list_square_sizes)
plt.title(f'trigger size vs ROC (all models)')
plt.show()

plt.figure(figsize=(10, 6)).patch.set_color('white')
plt.errorbar(list_square_sizes, xent_scores, xent_err, marker='o', label='all models')
plt.grid()
plt.legend()
plt.xlabel('square trigger size')
plt.ylabel('Cross-Entropy for SVM classifier')
plt.xticks(list_square_sizes)
plt.title(f'trigger size vs Cross-Entropy (all models)')
plt.show()

# Plot trigger-size vs ROC using 5-fold cross validation for Filters data

In [ ]:
list_filters = ['gotham', 'kelvin', 'lomo', 'nashville', 'toaster']
dict_filter_indexes = {size:index*2 for index, size in enumerate(list_filters)}
roc_auc_scores, roc_auc_err = [], []
xent_scores, xent_err = [], []
for filter_name, index in dict_filter_indexes.items():
    path = 'confusion-reports/ics_svm/round2-train-dataset/round2-train-dataset_filters.csv'
    X, y = read_features(path, filter_name)
    
    print(f'selecting columns {index} and {index+1}')
    X = X[:, index:index+2]
    
    skf = StratifiedKFold(n_splits=10, shuffle=True)

    cv_scores_roc, cv_scores_xent = [], []
    for i, (train_index, test_index) in enumerate(skf.split(X, y)):
        X_train, y_train = X[train_index, :], y[train_index] # square_diffs have indexes 0,1 !!!
        X_test, y_test = X[test_index, :], y[test_index] # square_diffs have indexes 0,1 !!!
        roc, xent = evaluate_classifier(X_train, y_train, X_test, y_test)
        cv_scores_roc.append(roc)
        cv_scores_xent.append(xent)
    mean_roc, std_roc = np.mean(cv_scores_roc), np.std(cv_scores_roc)
    mean_xent, std_xent = np.mean(cv_scores_xent), np.std(cv_scores_xent)
    print(f'size={size}, index={index}: mean_roc={mean_roc}, std_roc={std_roc}, mean_xent={mean_xent}, std_xent={std_xent}\n')
    
    roc_auc_scores.append(mean_roc)
    roc_auc_err.append(std_roc)
    xent_scores.append(mean_xent)
    xent_err.append(std_xent)

plt.figure(figsize=(10, 6)).patch.set_color('white')
plt.errorbar(list_filters, roc_auc_scores, roc_auc_err, marker='o', label='all models')
plt.grid()
plt.legend()
plt.xlabel('square trigger size')
plt.ylabel('ROC AUC for SVM classifier')
plt.xticks(list_filters)
plt.title(f'trigger size vs ROC (all models)')
plt.show()

plt.figure(figsize=(10, 6)).patch.set_color('white')
plt.errorbar(list_filters, xent_scores, xent_err, marker='o', label='all models')
plt.grid()
plt.legend()
plt.xlabel('square trigger size')
plt.ylabel('Cross-Entropy for SVM classifier')
plt.xticks(list_filters)
plt.title(f'trigger size vs Cross-Entropy (all models)')
plt.show()

# merge or simplify datasets

In [ ]:
def select_columns_from_dataset():
    df = pd.read_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square20-gotham-kelvin-lomo-nashville-toaster.csv')
    for col_to_delete in ['square_mean_diff', 'square_std_diff', 'square_mean', 'square_std']:
        del df[col_to_delete]
    df.to_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_filters.csv', index=False)
    print('done')

def merge_datasets():
    df1 = pd.read_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_filters.csv')
    df2 = pd.read_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_squares.csv')
    for col in ['square25_mean_diff', 'square25_std_diff', 'square25_mean', 'square25_std']:
        df1[col] = df2[col]
    
#     series_data = []
#     for i in range(len(df1)):
#         trigger_type = df1.iloc[i]['trigger_type']
#         trigger_type_option = df1.iloc[i]['trigger_type_option']
#         if trigger_type == 'instagram':
#             series_data.append(trigger_type_option.replace('XForm', '').replace('Filter', ''))
#         else:
#             if trigger_type == 'None':
#                 series_data.append(trigger_type)
#             else:
#                 series_data.append(f'{trigger_type}-{trigger_type_option}')
#     df1['trigger_type_aux'] = series_data
    
    new_order_for_columns = [
        'model_name', 'model_architecture', 'model_label',
        'trigger_type_aux', # 'trigger_type', 'trigger_type_option',

        'square25_mean_diff', 'square25_std_diff',
        'gotham_mean_diff', 'gotham_std_diff',
        'kelvin_mean_diff', 'kelvin_std_diff',
        'lomo_mean_diff', 'lomo_std_diff',
        'nashville_mean_diff', 'nashville_std_diff',
        'toaster_mean_diff', 'toaster_std_diff',

        'clean_mean', 'clean_std',
        'square25_mean', 'square25_std',
        'gotham_mean', 'gotham_std',
        'kelvin_mean', 'kelvin_std',
        'lomo_mean', 'lomo_std',
        'nashville_mean', 'nashville_std',
        'toaster_mean', 'toaster_std',

        'trigger_color', 'num_classes',
    ]
    df1 = df1[new_order_for_columns]
    df1.to_csv(r'confusion-reports\ics_svm\round2-train-dataset\round2-train-dataset_square25-filters.csv', index=False)
    print('done')
merge_datasets()
# select_columns_from_dataset()